In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

### Data Collection

In [2]:
df_train = pd.read_csv("module5_course_handling_duplicate_train.csv")
df_test = pd.read_csv("module5_course_handling_duplicate_train.csv")

## Data Analysis

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Identify duplicates
# Adding a new column that flags duplicates based on all columns
# Identify duplicates based on all columns or specific columns
df['is_duplicate'] = df.duplicated(keep=False)

# Count of duplicates
print("Total duplicates:", df['is_duplicate'].sum())

# Visualization of duplicates
plt.figure(figsize=(10, 6))
sns.countplot(x='is_duplicate', data=df)
plt.title('Duplicate Records in Dataset')
plt.xlabel('Is Duplicate')
plt.ylabel('Count')
plt.show()

# Fixing the scatter plot error by removing 'hue' and 'style' that caused the problem
plt.figure(figsize=(12, 6))
plt.scatter(x=df.index, y=df['is_duplicate'].map({True: 1, False: 0}), c=['red' if x else 'blue' for x in df['is_duplicate']], marker='o')
plt.title('Distribution of Duplicates in Dataset')
plt.xlabel('Index')
plt.ylabel('Is Duplicate')
plt.yticks([0, 1], ['False', 'True'])
plt.grid(True)
plt.show()

# Explore duplicates related to 'Date' and 'Station_ID', especially on rainy days
# Define rainy days as days with Precipit greater than a threshold, e.g., 0.1mm
rainy_days_threshold = 0.1
df['rainy_day'] = df['Precipit'] > rainy_days_threshold

# Identify duplicates by 'Date' and 'Station_ID'
df['date_station_duplicate'] = df.duplicated(subset=['Date', 'station_id'], keep=False)

# Visualization of date and station duplicates, especially on rainy days
plt.figure(figsize=(10, 6))
sns.countplot(x='date_station_duplicate', hue='rainy_day', data=df)
plt.title('Duplicate Records by Date and Station on Rainy Days')
plt.xlabel('Is Duplicate by Date and Station')
plt.ylabel('Count')
plt.legend(title='Rainy Day')
plt.show()

# Cleaning up by removing the added columns after analysis
df.drop(columns=['is_duplicate', 'rainy_day', 'date_station_duplicate'], inplace=True)

NameError: name 'df' is not defined

## Data Cleaning - handling duplicate

In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

def prepare_data(df, drop_cols, target):
    """
    Prepare the dataset by dropping specified columns and splitting into features and target.
    
    Parameters:
        df (DataFrame): The DataFrame to process.
        drop_cols (list): List of columns to drop from the DataFrame.
        target (str): The target variable name.

    Returns:
        X (DataFrame): Features DataFrame.
        y (Series): Target variable Series.
    """
    df_ = df.drop(drop_cols, axis=1)
    X = df_.drop(target, axis=1)
    y = df_[target]
    return X, y

def train_and_evaluate(X_train, y_train, X_test, y_test):
    """
    Train a Linear Regression model and evaluate it using mean absolute error.
    
    Parameters:
        X_train (DataFrame): Training features.
        y_train (Series): Training target.
        X_test (DataFrame): Testing features.
        y_test (Series): Testing target.

    Returns:
        float: The mean absolute error of the predictions.
    """
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

# Handling duplicates with different strategies
# Strategy 1: No handling of duplicates
X_train, y_train = prepare_data(df_train, ['Date', 'id'], 'Precipit')
X_test, y_test = prepare_data(df_test, ['Date', 'id'], 'Precipit')
mae_no_duplicates_handled = train_and_evaluate(X_train, y_train, X_test, y_test)

# Strategy 2: Drop all duplicates
df_train_deduped = df_train.drop_duplicates()
X_train, y_train = prepare_data(df_train_deduped, ['Date', 'id'], 'Precipit')
X_test, y_test = prepare_data(df_test, ['Date', 'id'], 'Precipit')
mae_all_duplicates_dropped = train_and_evaluate(X_train, y_train, X_test, y_test)

# Strategy 3: Drop duplicates based on 'Date' only
df_train_date_deduped = df_train.drop_duplicates(subset=['Date'])
X_train, y_train = prepare_data(df_train_date_deduped, ['Date', 'id'], 'Precipit')
X_test, y_test = prepare_data(df_test, ['Date', 'id'], 'Precipit')
mae_date_duplicates_dropped = train_and_evaluate(X_train, y_train, X_test, y_test)

# Print mean absolute errors for each strategy
print(f"MAE without handling duplicates: {mae_no_duplicates_handled}")
print(f"MAE with all duplicates dropped: {mae_all_duplicates_dropped}")
print(f"MAE with date-based duplicates dropped: {mae_date_duplicates_dropped}")


MAE without handling duplicates: 0.5040567558693914
MAE with all duplicates dropped: 0.46979054660143105
MAE with date-based duplicates dropped: 0.40808520627546363
